In [4]:

import multiprocessing
import itertools
import pandas as pd
import os
import mysql.connector

from mysql.connector import Error
from tqdm import tqdm
from pprint import pprint
from multiprocessing import Process


In [5]:
# %pip install mysql-connector-python

In [6]:

# %pip install Unidecode nltk 

In [7]:

import nltk
import unidecode
import operator

from nltk.corpus import stopwords
from difflib import SequenceMatcher


def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [33]:


def get_conn():
    return mysql.connector.connect(
            host='192.168.0.217',
            database='propiedades',
            user='root',
            password='propiedades')

def test_connection():
    try:
        connection = get_conn()
        if connection.is_connected():
            db_Info = connection.get_server_info()
            print("Connected to MySQL Server version ", db_Info)
            cursor = connection.cursor()
            cursor.execute("select database();")
            record = cursor.fetchone()
            print("You're connected to database: ", record)
    except Error as e:
        print("Error while connecting to MySQL", e)
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")


test_connection()
connection = get_conn()
cursor = connection.cursor()


def read_all(query, cursor=cursor):
    cursor.execute(query)
    return cursor.fetchall()

def print_all(query, cursor=cursor):
    records = read_all(query, cursor)
    pprint(records)


Connected to MySQL Server version  8.0.26-google
You're connected to database:  ('propiedades',)
MySQL connection is closed


In [34]:


# unique_states = [('Tamaulipas',), ('México',), ('Sinaloa',), ('Campeche',), ('DF / CDMX',), ('Coahuila de Zaragoza',), ('Chihuahua',), ('Durango',), ('Jalisco',), ('Colima',), ('Veracruz de Ignacio de la Llave',), ('Guanajuato',), ('Puebla',), ('Sonora',), ('Oaxaca',), (None,), ('Chiapas',), ('Quintana Roo',), ('Michoacán de Ocampo',), ('Baja California Sur',), ('Aguascalientes',), ('Baja California',), ('Yucatán',), ('Nayarit',), ('Guerrero',), ('Morelos',), ('Nuevo León',), ('Querétaro',), ('Hidalgo',), ('Tlaxcala',), ('Tabasco',), ('San Luis Potosí',), ('Zacatecas',), ('Distrito Federal',)]
unique_states = read_all("SELECT DISTINCT state FROM sepomex", cursor)
unique_states.sort()
result = read_all('DESC sepomex', cursor)
fields = [t[0] for t in result]


In [35]:

# Permanently changes the pandas settings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# nltk.download("stopwords")
stopwords = stopwords.words('spanish')

In [36]:

def download_csv_from_mysql(cursor, unique_states, fields):
    n = len(unique_states)
    pbar = tqdm(enumerate(unique_states), total=n)
    for i, state in pbar:
        state = state[0]
        query = f"SELECT {', '.join(fields)} FROM sepomex WHERE state = '{state}'"
        res = read_all(query, cursor)
        df = pd.DataFrame(data=res)
        df.columns = fields
        df.to_csv(f"csv/{i}.csv", index=False)
        pbar.set_description(state)


# download_csv_from_mysql(cursor, unique_states, fields)

In [37]:

def get_df_from_sources(partition_fname):
    cols = ['id', 'colony']
    df_sql = pd.read_csv(partition_fname)
    # print(df_sql)
    values_sql = df_sql[~df_sql['colony'].isnull()][cols].values

    state_label = df_sql.loc[0,'state']

    df_web = pd.read_csv("./CPdescarga.csv", sep="|", encoding='utf8')
    subset = df_web[df_web["d_estado"] == state_label]
    cols = ['d_codigo', 'd_asenta']
    values_web = subset[cols].values

    return values_sql, values_web


In [30]:

def clean_str(s):
    s = s.lower()
    tokens = s.split(' ')
    new_s = [t for t in tokens if not t in stopwords]
    s = ' '.join(new_s)
    return unidecode.unidecode(s)

def get_data(values_sql, values_web):
    threshold = 0.8
    data = []  # idsql, idweb, new|update
    pbar = tqdm(values_sql, total=len(values_sql))
    for i, col_sql in pbar:
        sims = [(j, similar(col_sql, col_web), col_web) for j, col_web in values_web]
        sims.sort(key=operator.itemgetter(1), reverse=True)
        j, sim, col_web = sims[0]
        if sim == 0.0:  
            data.append([i, col_sql, "insert", sims[:5]])
        elif 1 > sim and sim > threshold:
            data.append([i, col_sql, "update", sims[:5]])
    return data


In [ ]:
# fnames

In [ ]:

def __load_colony_sims(fname):
    values_sql, values_web = get_df_from_sources("csv/" + fname)
    values_sql = [(i, clean_str(s)) for i, s in values_sql]
    values_web = [(i, clean_str(s)) for i, s in values_web]
    data = get_data(values_sql, values_web)
    df = pd.DataFrame(data, columns=['sql_id', 'colony_sql_name', 'action_string', 'knn_list'])
    df.to_csv("result/" + fname, index=False)

# fnames = os.listdir('csv')
def load_colony_sims(fnames, safe=True):
    n = len(fnames)
    for k, fname in enumerate(fnames):
        print("%d/%d in progress (%s)" % (k, n, fname))
        if safe:
            try:
                __load_colony_sims(fname)
            except Exception as e:
                print("[!] " + fname)
                print(e)
        else:
            __load_colony_sims(fname)


In [ ]:

def partitionate_source(src, cpu_count = multiprocessing.cpu_count()):
    # cpu_count = multiprocessing.cpu_count()
    split_count = len(src) // cpu_count
    groups = []
    for i in range(0, len(src), split_count):
        groups.append(fnames[i:i + split_count])
    return groups


fnames = os.listdir('csv')
# fnames = list(set(os.listdir('csv')) - set(os.listdir('result')))
fnames.sort()
groups = partitionate_source(fnames)

[['0.csv', '1.csv', '10.csv', '11.csv'],
 ['12.csv', '13.csv', '14.csv', '15.csv'],
 ['16.csv', '17.csv', '18.csv', '19.csv'],
 ['2.csv', '20.csv', '21.csv', '22.csv'],
 ['23.csv', '24.csv', '25.csv', '26.csv'],
 ['27.csv', '28.csv', '29.csv', '3.csv'],
 ['30.csv', '31.csv', '4.csv', '5.csv'],
 ['6.csv', '7.csv', '8.csv', '9.csv']]

In [ ]:

# procs = [Process(target=load_colony_sims, args=(g,)) for g in groups]
# for p in procs:
#     p.start()
# for p in procs:
#     p.join()

,id,colony,city,zipcode,state,state_id,state_url,state_label,city_id,lat,...,should_visit,history_image,historic_place_image,should_visit_image,cover_image,metro_rail,bus_routes,rtp_routes,trolley_routes,roads
0,2099,Zona Centro,Aguascalientes,20000.0,Aguascalientes,2,aguascalientes,Aguascalientes,17.0,21.885224,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2100,Delegación de La Secretaria de Comercio y Fome...,Aguascalientes,20008.0,Aguascalientes,2,aguascalientes,Aguascalientes,17.0,21.896055,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2101,Palacio de Gobierno del Estado de Aguascalientes,Aguascalientes,20009.0,Aguascalientes,2,aguascalientes,Aguascalientes,17.0,21.880075,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2102,Las Brisas,Aguascalientes,20010.0,Aguascalientes,2,aguascalientes,Aguascalientes,17.0,21.897366,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2103,Colinas del Rio,Aguascalientes,20010.0,Aguascalientes,2,aguascalientes,Aguascalientes,17.0,21.896146,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1306,138361,NaN,NaN,NaN,Aguascalientes,2,aguascalientes,Aguascalientes,NaN,21.900627,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1307,138400,Los Fresnos,Aguascalientes,20328.0,Aguascalientes,2,aguascalientes,Aguascalientes,17.0,21.911269,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1308,139145,Tahona Residencial,Aguascalientes,20218.0,Aguascalientes,2,aguascalientes,Aguascalientes,17.0,21.857842,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1309,139148,Quinta los Olivos,Aguascalientes,20326.0,Aguascalientes,2,aguascalientes,Aguascalientes,17.0,21.901316,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:

sql_cols=['colony', 'id', 'zipcode']
web_cols=['d_asenta', 'd_codigo', 'd_CP']


def get_df_from_sources(fname, sql_cols, web_cols):
    df_sql = pd.read_csv(fname, dtype={c: str for c in sql_cols})
    # print(df_sql)
    values_sql = df_sql[~df_sql['colony'].isnull()][sql_cols].values

    state_label = df_sql.loc[0,'state']

    df_web = pd.read_csv("./CPdescarga.csv", sep="|", encoding='utf8', dtype={c: str for c in web_cols})
    subset = df_web[df_web["d_estado"] == state_label]
    values_web = subset[web_cols].values
    return values_sql, values_web


In [70]:
import numpy as np


bname = '0.csv'
values_sql, values_web = get_df_from_sources(f"./csv/{bname}", sql_cols, web_cols)
values_sql = [(clean_str(s), *_) for s, *_ in values_sql]
values_web = [(clean_str(s), *_) for s, *_ in values_web]

In [71]:

threshold = 0.9
matches = set()
pbar = tqdm(values_sql)
for row_sql in pbar:
    s = row_sql[0]
    sims = [similar(s, w) for w, *_ in values_web]
    k = np.argmax(sims)
    if sims[k] > threshold:
        matches.add((row_sql, values_web[k]))
print(len(matches))

100%|██████████| 1299/1299 [00:34<00:00, 37.52it/s]

1070


In [72]:
["NULL"] * 3

['NULL', 'NULL', 'NULL']

In [73]:
data = []
for s, w in matches:
    data.append((*s, *w))
    values_sql.remove(s)
    if w in values_web:
        values_web.remove(w)
print(len(values_sql))
for s in values_sql:
    data.append((*s, "NULL", "NULL", "NULL"))
print(len(values_web))
for w in values_web:
    data.append(("NULL", "NULL", "NULL", w))

229
450


In [74]:
df = pd.DataFrame(data=data, columns=sql_cols + web_cols)
df.to_csv(bname, index=False)

In [75]:
df.to_excel("0.xlsx", index=False)